In [7]:
# coding: UTF-8
import argparse
import logging
import os
import random
import sys
from datetime import datetime
from posixpath import join
sys.path.append('..')
import faiss
import numpy as np
import pandas as pd
# using labse
# from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from torch.nn import *

from loader.DBP15KRawNeighbors import DBP15KRawNeighbors
from script.preprocess.deal_raw_dataset import MyRawdataset
from settings import *

sys.argv=['']
del sys

### Trainer

1. link_loader:  
mode:选择子数据集

In [ ]:
link = {}
mode = 'zh_en'
f = 'test.ref'
link_data = pd.read_csv(join(join(DATA_DIR, 'DBP15K', mode), f), sep='\t', header=None)
link_data.columns = ['entity1', 'entity2']
entity1_id = link_data['entity1'].values.tolist()
entity2_id = link_data['entity2'].values.tolist()
for i, _ in enumerate(entity1_id):
    link[entity1_id[i]] = entity2_id[i]
    link[entity2_id[i]] = entity1_id[i]

link_data['entity1']: pd column  
link_data['entity1'].values: np.array  
link{}: 保存的是zh->en和en->zh的双倍link  

In [8]:
len(link)

21000

if training:
self._model.update(self.model)之后  
会执行self._model.eval() 

2. save_model
torch.save(model.state_dict(), path) 保存模型，但是这里传入的是model? torch.save也可以保存model

3. evaluate
with torch.no_grad():
model.eval() 这两句取消了所有梯度计算和反向传播

for idx, item in enumerate(dataloader) dataloader返回的是idx和具体的数据

idx->[19388, 1]
x->[19388, 20, 788]
y->[19388, 1]

In [10]:
from model.layers_LaBSE_neighbor import *
parser = argparse.ArgumentParser()
args = parse_options(parser)
model = MyEmbedder(args, VOCAB_SIZE)
loader1 = DBP15KRawNeighbors(args.language, "1")
myset1 = MyRawdataset(loader1.id_neighbors_dict, loader1.id_adj_tensor_dict)
eval_loader1 = Data.DataLoader(
            dataset=myset1,  # torch TensorDataset format
            batch_size=args.batch_size,  # all test data
            shuffle=True,
            drop_last=False,
        )

In [11]:
ids_1, ids_2, vector_1, vector_2 = list(), list(), list(), list()
inverse_ids_2 = dict()
with torch.no_grad():
    model.eval()
    for sample_id_1, (token_data_1, id_data_1) in tqdm(enumerate(eval_loader1)):
        entity_vector_1 = model(token_data_1).squeeze().detach().cpu().numpy()
        ids_1.extend(id_data_1.squeeze().tolist())
        vector_1.append(entity_vector_1)

    for sample_id_2, (token_data_2, id_data_2) in tqdm(enumerate(eval_loader2)):
        entity_vector_2 = model(token_data_2).squeeze().detach().cpu().numpy()
        ids_2.extend(id_data_2.squeeze().tolist())
        vector_2.append(entity_vector_2)

0it [00:24, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument mat2 in method wrapper_bmm)